In [1]:
import numpy as np

# from Input_Variables.read_vars import raw_data_storage, \
#                                       mfdfa_wins, mfdfa_q_list, mfdfa_rev_seg, mfdfa_pol_order, \
#                                       fpca_min_time

from Input_Variables.read_vars import raw_data_storage, \
                                      daily_stats_features_lower, daily_stats_features_upper

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.daily_stats import Daily_Stats_Features
# from Feature_Generation.time_series_feature_creation import TS_Feature_Creation

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
daily_stats_features=Daily_Stats_Features()


# PySpark

### PySpark: Reading In Data

In [3]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 23:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### PySpark: Sklearn Pipeline in PySpark

In [4]:
####### PySpark
pyspark_transform_schema=pandas_udf_data_schema.sklearn_pyspark_schema()
pyspark_transformations=pandas_sklearn_pipeline.pyspark_sklearn_pipeline(df=pyspark_df, 
                                                                         output_schema=pyspark_transform_schema)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [5]:
pyspark_transformations.head(2)

23/02/26 23:20:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/02/26 23:20:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expec

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

[Row(PatientId='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=', Value=-0.8552401065826416, GlucoseDisplayTime=datetime.datetime(2022, 9, 13, 19, 15, 45), GlucoseDisplayDate=datetime.date(2022, 9, 13), inserted=[1, 0], missing=[1, 0]),
 Row(PatientId='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=', Value=-0.8820019364356995, GlucoseDisplayTime=datetime.datetime(2022, 9, 13, 19, 20, 45), GlucoseDisplayDate=datetime.date(2022, 9, 13), inserted=[1, 0], missing=[1, 0])]

### PySpark: Features: Daily Stats

In [6]:
####### PySpark
pyspark_features_daily_stats_schema=pandas_udf_data_schema.daily_stats_schema()

pyspark_features_daily_stats=daily_stats_features.pyspark_sklearn_pipeline(df=pyspark_transformations, 
                                                                           output_schema=pyspark_features_daily_stats_schema, 
                                                                           lower=daily_stats_features_lower, 
                                                                           upper=daily_stats_features_upper)

In [7]:
pyspark_features_daily_stats.head(2)

23/02/26 23:20:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: PostDate, IngestionDate, PostId, PostTime, PatientId, Stream, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
 Schema: PostDate, IngestionDate, PostID, PostTime, PatientId, Stram, SequenceNumber, TransmitterNumber, ReceiverNumber, RecordedSystemTime, RecordedDisplayTime, RecordedDisplayTimeRaw, TransmitterId, TransmitterTime, GlucoseSystemTime, GlucoseDisplayTime, GlucoseDisplayTimeRaw, Value, Status, TrendArrow, TrendRate, IsBackFilled, InternalStatus, SessionStartTime
Expected: Stram but found: Stream
CSV file: file:///Users/carlosmonsivais/Desktop/capstone_data/ahr414_glucose_sample%20-%20ahr414_glucose_sample.csv


/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

[Row(PatientId='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=', Value=-0.8552401065826416, GlucoseDisplayTime=datetime.datetime(2022, 9, 13, 19, 15, 45), GlucoseDisplayDate=datetime.date(2022, 9, 13), inserted=[1, 0], missing=[1, 0], Median=0.3891858756542206, Mean=0.31124788522720337, Std Dev=0.597746729850769, Max=1.0849939584732056, Min=-0.8820019364356995, AreaBelow=3972.259033203125, AreaAbove=0.0),
 Row(PatientId='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=', Value=-0.8820019364356995, GlucoseDisplayTime=datetime.datetime(2022, 9, 13, 19, 20, 45), GlucoseDisplayDate=datetime.date(2022, 9, 13), inserted=[1, 0], missing=[1, 0], Median=0.3891858756542206, Mean=0.31124788522720337, Std Dev=0.597746729850769, Max=1.0849939584732056, Min=-0.8820019364356995, AreaBelow=3972.259033203125, AreaAbove=0.0)]

# Pandas

### Pandas: Reading in Data

In [8]:
####### Pandas
pandas_df=reading_data.read_in_pandas()

### Pandas: Sklearn Pipeline in Pandas

In [9]:
####### Pandas
pandas_transformations=pandas_sklearn_pipeline.pandas_transform_features(df=pandas_df)

/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inserted'] = 0
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['missing'].iloc[missing_label] = 1
/Users/carlosmonsivais/Desktop/glucose-data-analysis/Data_Pipeline/fill_missing_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [10]:
pandas_transformations.head(2)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,-0.85586,2022-09-13 19:15:45,2022-09-13,"[1.0, 0.0]","[1.0, 0.0]"
1,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,-0.882658,2022-09-13 19:20:45,2022-09-13,"[1.0, 0.0]","[1.0, 0.0]"


### Pandas: Features: Daily Stats

In [11]:
pandas_features_daily_stats=daily_stats_features.pandas_compressDailyValues(data=pandas_transformations, 
                                                                            lower=daily_stats_features_lower, 
                                                                            upper=daily_stats_features_upper)

In [12]:
pandas_features_daily_stats.head(2)

,PatientId,Value,GlucoseDisplayTime,GlucoseDisplayDate,inserted,missing,Median,Mean,Std Dev,Max,Min,AreaBelow,AreaAbove
0,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,-0.85586,2022-09-13 19:15:45,2022-09-13,"[1.0, 0.0]","[1.0, 0.0]",0.390258,0.312214,0.598559,1.087011,-0.882658,3972.203824,0
1,tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=,-0.882658,2022-09-13 19:20:45,2022-09-13,"[1.0, 0.0]","[1.0, 0.0]",0.390258,0.312214,0.598559,1.087011,-0.882658,3972.203824,0


### Not Ready, Do Not Run
### Need to add Summary statistics to Features
### Multifractal Detrended Fluctuation Analysis, Poincare Analysis, Functional Principal Component Analysis, Entropy

In [ ]:
ts_feature_creation=TS_Feature_Creation()

# Need to import variables from file
mfdfa=ts_feature_creation.mfdfa_extraction(data=pandas_transformations,
                                   win_sizes=mfdfa_wins,
                                   q_list=mfdfa_q_list,
                                   rev_seg=mfdfa_rev_seg,
                                   pol_order=mfdfa_pol_order)
print(mfdfa)

poincare=ts_feature_creation.poincare_extraction(data=pandas_transformations)
print(poincare)

# Fuction Works
entropy=ts_feature_creation.entropy_extraction(data=pandas_transformations)
print(entropy)

### Modeling

### TensorFlow Time Series Probability Models: [TensorFlow](https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand)
### PySpark XGBoost: [PySpark](https://xgboost.readthedocs.io/en/stable/tutorials/spark_estimator.html)